# Refinement Procedure
for each dataset, do the following refinement process. it doesnt have to be executed in listed order.
1. Process miscellaneous things.
2. unit conversion
3. outlier deletion

In [32]:
import pandas as pd
from warnings import simplefilter
from tqdm import tqdm
import numpy as np
import datetime
import sys
simplefilter(action="ignore", category=pd.errors.DtypeWarning)
pd.set_option('display.max_columns', None)

dirs = [str('processed_data/sepsis/%s_SCR.csv' %i) for i in ['omr', 'LE', 'CE', 'IE', 'OE', 'PE']]

def inch_to_cm(x):
    x = np.array(x).astype('float')

    return x*2.54

def lbs_to_kg(x):
    x = np.array(x).astype('float')

    return x/2.205

def F_to_C(x):
    x = np.array(x).astype('float')

    return (x-32)*5/9

def mcg_to_mg(x):
    x = np.array(x).astype('float')

    return x/1000

## 1. omr processing

In [3]:
df = pd.read_csv(dirs[0])
# date to time
# for avoding information leakage, time of dates in omr data is assumed as the last time(23:59:59) of the date.
df.chartdate = pd.to_datetime(df.chartdate + ' 23:59:59')
df.columns = [i if idx!=1 else 'charttime' for idx, i in enumerate(df.columns)]

In [4]:
# we will use only the information of the selected cohorts' admission
admissions = pd.read_csv('hosp/admissions.csv')
admissions.admittime = pd.to_datetime(admissions.admittime) 
admissions.dischtime = pd.to_datetime(admissions.dischtime) 
icustays = pd.read_csv('processed_data/sepsis/icustays_wsusinf.csv')
subjectid = df.subject_id.unique().tolist()

include_idx = []
for sbj_id in tqdm(subjectid):
    tmp_cond = (icustays.suspected_infection == 1) & (icustays.subject_id == sbj_id)
    adm_id = icustays.loc[tmp_cond].hadm_id.tolist()
    
    ATDT = admissions.loc[admissions.hadm_id.isin(adm_id), ('admittime', 'dischtime')].reset_index(drop=True)
    
    for i in range(ATDT.shape[0]):
        tmp_cond = (df.subject_id == sbj_id) & (df.charttime >= ATDT.loc[i].tolist()[0]) & (df.charttime <= ATDT.loc[i].tolist()[1])
        include_idx += tmp_cond.index[tmp_cond].tolist()
df = df.iloc[include_idx].reset_index(drop=True)

100%|██████████| 12639/12639 [02:14<00:00, 93.99it/s] 


In [5]:
# missing value handling
print(df.loc[pd.isna(df.result_value)].result_name.value_counts())
df = df.drop(pd.isna(df.result_value).index[pd.isna(df.result_value)]).reset_index(drop=True)

Series([], Name: result_name, dtype: int64)


In [6]:
# leave only the latest item,  if two same items exists at the same time
drop_idx = []
for sbj in tqdm(df.subject_id.unique()):
    tmp_cond = df.subject_id == sbj
    part_df = df.loc[tmp_cond]

    for ct in part_df.charttime.unique():
        tmp_cond = (part_df.charttime == ct)
        part_df_2 = part_df.loc[tmp_cond]

        for rn in part_df_2.result_name.unique():
            tmp_cond = (part_df_2.result_name == rn)
            if sum(tmp_cond) > 1:
                drop_idx += [i for i in tmp_cond.index[tmp_cond] if i != tmp_cond.index[tmp_cond][-1]]

df = df.drop(drop_idx).reset_index(drop=True)

100%|██████████| 3797/3797 [00:03<00:00, 1248.97it/s]


In [7]:
# blood pressure processing
# the blood pressure value is wriiten as sbp/dbp.
# validate the sbp > dbp and split them to two rows.
tmp_data = []
tmp_cond = (df.result_name.str.contains('Blood'))

for idx in tqdm(tmp_cond[tmp_cond].index):
    sbj, ct = df.iloc[idx, 0:2]
    sbp, dbp = df.iloc[idx, -1].split('/')
    sbp, dbp = int(sbp), int(dbp) 

    if sbp > dbp:
        tmp_data.append([sbj, ct, 'SBP', sbp])
        tmp_data.append([sbj, ct, 'DBP', dbp])

tmp_data = pd.DataFrame(tmp_data, columns=[i for i in df.columns if i != 'seq_num'])

100%|██████████| 1146/1146 [00:00<00:00, 8293.82it/s]


In [8]:
# replace blood pressure data to new one
# and also drop eGFR, BMI
tmp_cond = (df.result_name.str.contains('Blood|eGFR|BMI'))
df = df.drop(tmp_cond[tmp_cond].index)

df = pd.concat([df.loc[:, [i for i in df.columns if i != 'seq_num']], tmp_data], axis=0).sort_values(['subject_id', 'charttime']).reset_index(drop=True)

In [9]:
# check if df has result value that can not be type conversed to float.
drop_idx = []
for i in df.index:
    try:
        tmp = float(df.iloc[i, -1])
    except:
        drop_idx.append(i)
print(df.iloc[drop_idx])
df = df.drop(drop_idx).reset_index(drop=True) ##### really important

Empty DataFrame
Columns: [subject_id, charttime, result_name, result_value]
Index: []


In [10]:
# now we do unit conversion for OMR
tmp_cond = (df.result_name.str.contains('Inches'))
df.loc[tmp_cond, 'result_value'] = inch_to_cm(df.loc[tmp_cond, 'result_value'])
df.loc[tmp_cond, 'result_name'] = 'Height'

tmp_cond = (df.result_name.str.contains('Lbs'))
df.loc[tmp_cond, 'result_value'] = lbs_to_kg(df.loc[tmp_cond, 'result_value'])
df.loc[tmp_cond, 'result_name'] = 'Weight'

In [11]:
df.result_name.value_counts()

Weight    3944
Height    2098
SBP       1146
DBP       1146
Name: result_name, dtype: int64

In [12]:
# value distribution describe
df.result_value = df.result_value.astype('float')
df.groupby('result_name').result_value.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

,count,mean,std,min,0.1%,1%,5%,50%,95%,99%,99.9%,max
result_name,,,,,,,,,,,,
DBP,1146.0,69.109948,13.885878,28.0,29.14500,40.000000,47.000000,69.00000,92.000000,102.550000,113.855000,120.000000
Height,2098.0,167.287294,16.222356,0.0,0.49276,137.083800,152.400000,167.64000,184.150000,190.500000,198.120000,449.580000
SBP,1146.0,120.684119,22.614476,52.0,60.87000,73.450000,87.000000,119.00000,159.000000,184.000000,221.275000,226.000000
Weight,3944.0,80.020535,25.435669,0.0,0.00000,36.527438,48.075283,76.78458,124.235147,154.416009,225.543946,550.195011


In [13]:
# weight and height, will be concat into one df and then the outlier criteria will determined.
# for DBP, 10<=dbp<=175 were used 
# for SBP, 0<SBP<=300 were used
drop_idx = []
drop_bp_idx = []
# Height                                    
tmp_cond = ((df.result_name == 'Height') & ((df.result_value < 0) | (df.result_value > 240)))
drop_idx += tmp_cond.index[tmp_cond].to_list()
drop_bp_idx += tmp_cond.index[tmp_cond].to_list()

# Weight
tmp_cond = ((df.result_name == 'Weight') & ((df.result_value < 0) | (df.result_value > 250)))
drop_idx += tmp_cond.index[tmp_cond].to_list()
drop_bp_idx += tmp_cond.index[tmp_cond].to_list()

# SBP
tmp_cond = ((df.result_name == 'SBP') & ((df.result_value <= 0) | (df.result_value > 300)))
drop_idx += tmp_cond.index[tmp_cond].to_list()
drop_bp_idx += tmp_cond.index[tmp_cond].to_list()

# SBP
tmp_cond = ((df.result_name == 'SBP') & ((df.result_value <= 0) | (df.result_value > 300)))
drop_idx += tmp_cond.index[tmp_cond].to_list()
drop_bp_idx += tmp_cond.index[tmp_cond].to_list()

# DBP
tmp_cond = ((df.result_name == 'DBP') & ((df.result_value < 10) | (df.result_value > 175)))
drop_idx += tmp_cond.index[tmp_cond].to_list()
drop_bp_idx += tmp_cond.index[tmp_cond].to_list()

# corresponding SBP, DBP deletion
for idx in tqdm(drop_bp_idx):
    sbj, ct = df.iloc[idx, 0:2]

    tmp_cond = (df.subject_id == sbj) & (df.charttime == ct) & (df.result_name.isin(['SBP', 'DBP']))
    drop_idx += tmp_cond.index[tmp_cond].to_list()

df.drop(drop_idx)# 4 rows were deleted

100%|██████████| 4/4 [00:00<00:00, 1002.40it/s]


,subject_id,charttime,result_name,result_value
0,10001884,2131-01-07 23:59:59,Weight,68.689342
1,10003400,2137-02-24 23:59:59,Weight,83.764172
2,10003400,2137-08-05 23:59:59,Weight,97.505669
3,10007818,2146-06-10 23:59:59,Height,185.420000
4,10007818,2146-06-10 23:59:59,Weight,89.183673
...,...,...,...,...
8329,19996783,2188-05-10 23:59:59,Weight,60.770975
8330,19997367,2127-04-02 23:59:59,Weight,55.487528
8331,19997473,2173-09-11 23:59:59,Height,158.750000
8332,19997473,2173-09-11 23:59:59,Weight,64.489796


In [15]:
# delete outliers and save the refined dataframe
df.drop(drop_idx).to_csv('processed_data/sepsis/omr_R.csv', index=False)

## 2. labevents processing

In [2]:
df = pd.read_csv(dirs[1])
df.charttime = pd.to_datetime(df.charttime) # for time should be able to calculated

In [3]:
# from the d_predictor, choose the item ids for labevents
d_predictor = pd.read_csv('processed_data/sepsis/d_predictors.csv')
tmp = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()]
itemids_LE = []
for i in tmp: 
    itemids_LE += i

In [4]:
# filter data to have only predictor relevant rows and relevent columns
tmp_cond = df.itemid.isin(itemids_LE)
df = df.loc[tmp_cond, ('subject_id', 'hadm_id', 'itemid', 'charttime', 'storetime', 'valuenum', 'valueuom')].reset_index(drop=True)

In [5]:
# check if df has result value that can not be type conversed to float.
# float(df.iloc[:, -2])
drop_idx = []
for i in tqdm(df.index):
    try:
        tmp = float(df.iloc[i, -2])
    except:
        drop_idx.append(i)
print(df.iloc[drop_idx])
df = df.drop(drop_idx).reset_index(drop=True) ##### really important

100%|██████████| 4000535/4000535 [01:02<00:00, 64457.61it/s]


Empty DataFrame
Columns: [subject_id, hadm_id, itemid, charttime, storetime, valuenum, valueuom]
Index: []


In [6]:
# missing value handling
print(df.loc[pd.isna(df.valuenum)].itemid.value_counts())
df = df.drop(pd.isna(df.valuenum).index[pd.isna(df.valuenum)]).reset_index(drop=True)

50828    24792
51265     1663
50885     1340
51301     1258
51221      965
51222      787
50889      394
50971      359
51006      350
50820      122
50983       85
50813       80
51108        2
50825        1
Name: itemid, dtype: int64


In [7]:
# unit conversion
# check which item may need unit conversion 
d_predictor['items'][~pd.isna(d_predictor.LE)].ravel()

array(['Temperature', 'Bilirubin', 'Platelets', 'Creatinine', 'Lactate',
       'BUN', 'Arterial pH', 'WBC', 'Hemoglobin', 'Hematocrit',
       'Potassium', 'Urine output', 'Sodium', 'C Reactive Protein',
       'Ventilator'], dtype=object)

In [8]:
# temperature
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][0]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# because it is not written in dataset, we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])
# we can think that the temperatuer unit is Celcius

Series([], Name: valueuom, dtype: int64)


count    71984.000000
mean        37.030992
std          1.029730
min          0.000000
0.1%        32.000000
1%          33.900000
5%          35.600000
50%         37.000000
95%         38.600000
99%         39.400000
99.9%       40.300000
max         57.000000
Name: valuenum, dtype: float64

In [9]:
# bilirubin
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][1]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mg/dL    157420
Name: valueuom, dtype: int64


count    157420.000000
mean          4.165599
std           7.750764
min           0.000000
0.1%          0.100000
1%            0.200000
5%            0.200000
50%           1.100000
95%          21.300000
99%          38.800000
99.9%        56.600000
max          87.200000
Name: valuenum, dtype: float64

In [10]:
# platelets
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][2]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

K/uL    439772
Name: valueuom, dtype: int64


count    439772.000000
mean        218.966494
std         158.907163
min           5.000000
0.1%          6.000000
1%           12.000000
5%           30.000000
50%         190.000000
95%         516.000000
99%         739.000000
99.9%      1126.229000
max        2009.000000
Name: valuenum, dtype: float64

In [11]:
# creatinine
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][3]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mg/dL    55
Name: valueuom, dtype: int64


count    55.000000
mean      1.794545
std       1.348857
min       0.100000
0.1%      0.105400
1%        0.154000
5%        0.570000
50%       1.300000
95%       3.560000
99%       6.312000
99.9%     7.381200
max       7.500000
Name: valuenum, dtype: float64

In [12]:
# lactate
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][4]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mmol/L    177872
Name: valueuom, dtype: int64


count    1.778720e+05
mean     9.794278e+00
std      3.025737e+03
min      0.000000e+00
0.1%     4.000000e-01
1%       6.000000e-01
5%       8.000000e-01
50%      1.800000e+00
95%      7.600000e+00
99%      1.390000e+01
99.9%    2.100000e+01
max      1.276103e+06
Name: valuenum, dtype: float64

In [13]:
# BUN
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][5]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mg/dL    475361
Name: valueuom, dtype: int64


count    475361.000000
mean         32.570594
std          25.540428
min           0.000000
0.1%          2.000000
1%            4.000000
5%            7.000000
50%          25.000000
95%          85.000000
99%         121.000000
99.9%       169.000000
max         305.000000
Name: valuenum, dtype: float64

In [14]:
# arterial pH
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][6]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

units    285339
Name: valueuom, dtype: int64


count    285339.000000
mean          7.376181
std           0.093822
min           0.940000
0.1%          6.930000
1%            7.090000
5%            7.210000
50%           7.390000
95%           7.500000
99%           7.550000
99.9%         7.610000
max           7.960000
Name: valuenum, dtype: float64

In [15]:
# WBC
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][7]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

K/uL    430106
Name: valueuom, dtype: int64


count    430106.000000
mean         11.786931
std          21.262093
min           0.000000
0.1%          0.100000
1%            0.300000
5%            2.700000
50%          10.100000
95%          25.200000
99%          41.000000
99.9%       101.989500
max       12500.000000
Name: valuenum, dtype: float64

In [16]:
# hemoglobin
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][8]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

g/dL    462063
Name: valueuom, dtype: int64


count    462063.000000
mean          9.324117
std           1.798835
min           0.000000
0.1%          4.900000
1%            6.300000
5%            7.000000
50%           9.000000
95%          12.700000
99%          14.500000
99.9%        16.700000
max          98.000000
Name: valuenum, dtype: float64

In [17]:
# hematocrit
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][9]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

%    463919
Name: valueuom, dtype: int64


count    463919.000000
mean         28.501613
std           5.191310
min           2.100000
0.1%         15.500000
1%           19.400000
5%           21.600000
50%          27.700000
95%          38.300000
99%          43.900000
99.9%        50.708200
max          67.000000
Name: valuenum, dtype: float64

In [18]:
# potassium
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][10]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mEq/L    499588
Name: valueuom, dtype: int64


count    499588.000000
mean          4.108045
std           0.632763
min           1.000000
0.1%          2.500000
1%            2.900000
5%            3.200000
50%           4.000000
95%           5.200000
99%           6.000000
99.9%         7.900000
max          12.700000
Name: valuenum, dtype: float64

In [19]:
# urine output
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][11]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mL    561
Name: valueuom, dtype: int64


count      561.000000
mean      1760.192513
std       2616.904499
min          5.000000
0.1%         5.000000
1%           6.600000
5%         200.000000
50%       1400.000000
95%       3950.000000
99%       6430.000000
99.9%    30084.000000
max      55550.000000
Name: valuenum, dtype: float64

In [20]:
# sodium
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][12]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mEq/L    499423
Name: valueuom, dtype: int64


count    499423.000000
mean        138.681807
std           5.731934
min          67.000000
0.1%        116.000000
1%          125.000000
5%          130.000000
50%         139.000000
95%         148.000000
99%         154.000000
99.9%       162.000000
max         184.000000
Name: valuenum, dtype: float64

In [21]:
# CRP
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][13]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mg/L    4873
Name: valueuom, dtype: int64


count    4874.000000
mean       90.794019
std        77.993806
min         0.200000
0.1%        0.400000
1%          0.700000
5%          3.200000
50%        70.500000
95%       248.410000
99%       287.427000
99.9%     302.303500
max       586.600000
Name: valuenum, dtype: float64

In [22]:
# vent
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.LE[~pd.isna(d_predictor.LE)].ravel()][14]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# it is shown that unit conversion is not needed
# anyway we should check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

Series([], Name: valueuom, dtype: int64)


count    0.0
mean     NaN
std      NaN
min      NaN
0.1%     NaN
1%       NaN
5%       NaN
50%      NaN
95%      NaN
99%      NaN
99.9%    NaN
max      NaN
Name: valuenum, dtype: float64

In [23]:
# urine output outlier handling
admissions = pd.read_csv('hosp/admissions.csv')
admissions.admittime = pd.to_datetime(admissions.admittime)
admissions.dischtime = pd.to_datetime(admissions.dischtime)
raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains('urin'))&(~pd.isna(d_predictor.LE)), ('LE')].to_numpy()
raw_itemid = raw_itemid[0]
tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]

for adm in df.hadm_id.unique():
    tmp_cond = (df.hadm_id == adm) & (df.itemid.isin(tmp_itemid))
    part_df = df.loc[tmp_cond].reset_index(drop=True)
    tmp_cond = admissions.hadm_id == adm
    at, dt = admissions.loc[tmp_cond, ('admittime', 'dischtime')].to_numpy()[0]
    for idx, ct in enumerate(part_df.charttime):
        if idx == 0:
            td = (ct - at)/datetime.timedelta(hours=1)
            urine_hr = part_df.loc[idx, 'valuenum'] / (td + sys.float_info.epsilon)
        elif idx == part_df.index[-1]:
            td = (dt - ct)/datetime.timedelta(hours=1)
            urine_hr = part_df.loc[idx, 'valuenum'] / (td + sys.float_info.epsilon)
        else: 
            td = (ct - part_df.loc[idx-1, 'charttime'])/datetime.timedelta(hours=1)
            urine_hr = part_df.loc[idx, 'valuenum'] / (td + sys.float_info.epsilon)
        
        if urine_hr > 1000:
            print(f'{adm}, {ct}, {urine_hr}')
            tmp_cond = (df.hadm_id == adm) & (df.charttime == ct) & (df.itemid.isin(tmp_itemid))
            if td != 0:
                df.loc[tmp_cond, 'valuenum'] = 1000*td
            else:
                df.loc[tmp_cond, 'valuenum'] = 1000

26299168.0, 2171-11-10 12:00:00, 7.656119366529843e+18
22607104.0, 2167-10-21 00:43:00, 1.3510798882111488e+18
22036881.0, 2140-09-05 15:06:00, 1.7676628537429197e+19
20880376.0, 2128-07-28 13:12:00, 6.755399441055744e+18
20963681.0, 2177-04-28 19:46:00, 2.251799813685248e+18
24808859.0, 2188-10-11 18:06:00, 209999.9999999972
24808859.0, 2188-10-11 18:06:00, 1.5762598695796736e+19
24535091.0, 2124-05-22 08:00:00, 8.556839292003942e+18
25809889.0, 2132-12-29 03:54:00, 4.503599627370496e+16
26341885.0, 2130-03-12 06:00:00, 1.5762598695796736e+19
24976204.0, 2163-03-27 08:56:00, 4.503599627370496e+18
28494482.0, 2138-06-06 19:00:00, 5.62949953421312e+17
27199030.0, 2183-01-20 06:13:00, 2.7021597764222976e+18
27199030.0, 2183-01-25 08:36:00, 4.503599627370496e+18
24855743.0, 2187-10-08 13:49:00, 3.8280596832649216e+18
26250746.0, 2147-05-12 22:23:00, 6.530219459687219e+18
29186168.0, 2145-04-19 18:10:00, 2.9273397577908224e+18
21479712.0, 2115-01-07 16:24:00, 2.9273397577908224e+18
2374946

In [29]:
# outlier handling (except urine)
outlier_criteria = pd.read_csv('processed_data/sepsis/outlier_criteria.csv')
drop_idx = []
for item in outlier_criteria.features:
    raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains(item.lower()[0:4]))&(~pd.isna(d_predictor.LE)), ('LE')].to_numpy()
    if len(raw_itemid) != 0:
        raw_itemid = raw_itemid[0]
        tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
        print(f'{item}, {tmp_itemid}')
        
        lb, ub = outlier_criteria.loc[outlier_criteria.features.str.lower().str.contains(item.lower()[0:4]), ('lb', 'ub')].to_numpy()[0]
        if item.lower()[0:4] in [i[0:4] for i in outlier_criteria.loc[outlier_criteria.criteria == '<=, >=', 'features'].str.lower()]: # <=, >=
            tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.valuenum <= lb) | (df.valuenum >= ub)))
        elif item.lower()[0:4] in [i[0:4] for i in outlier_criteria.loc[outlier_criteria.criteria == '<, >=', 'features'].str.lower()]: # <, >=
            tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.valuenum < lb) | (df.valuenum >= ub)))
        elif item.lower()[0:4] in [i[0:4] for i in outlier_criteria.loc[outlier_criteria.criteria == '<=, >', 'features'].str.lower()]: # <=, >
            tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.valuenum <= lb) | (df.valuenum > ub)))
        elif item.lower()[0:4] in [i[0:4] for i in outlier_criteria.loc[outlier_criteria.criteria == '<, >', 'features'].str.lower()]: #<, >
            tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.valuenum < lb) | (df.valuenum > ub)))
        drop_idx += tmp_cond.index[tmp_cond].to_list()

print(f'{df.shape[0]} -> {df.iloc[list(set(df.index) - set(drop_idx))].shape[0]}')
df.iloc[drop_idx].itemid.value_counts()

Temperature, [50825]
Bilirubin, [50885, 53089]
Platelets, [51265, 51704]
Creatinine, [51081]
Lactate, [50813, 52442]
BUN, [51006, 52647]
Arterial pH, [50820]
WBC, [51300, 51301, 51755, 51756]
Hemoglobin, [50811, 51222, 51640]
Hematocrit, [51221, 51638, 51639, 52028]
Potassium, [50971, 52610]
Sodium, [50983, 52623]
3968337 -> 3968001


51301    136
50825     62
51265     55
50885     46
50811     15
50820     11
50971      4
50813      2
51006      2
51222      2
50983      1
Name: itemid, dtype: int64

In [30]:
df.drop(drop_idx).to_csv('processed_data/sepsis/LE_R.csv', index=False)

## 3. chartevents processing

In [2]:
df = pd.read_csv(dirs[2])
df.charttime = pd.to_datetime(df.charttime) # for time should be able to calculated

In [3]:
# from the d_predictor, choose the item ids for labevents
d_predictor = pd.read_csv('processed_data/sepsis/d_predictors.csv')
tmp = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()]
itemids_CE = []
for i in tmp: 
    itemids_CE += i

In [4]:
# filter data to have only predictor relevant rows and relevent columns
tmp_cond = df.itemid.isin(itemids_CE)
df = df.loc[tmp_cond, ('subject_id', 'hadm_id', 'stay_id', 'charttime', 'storetime', 'itemid', 'valuenum', 'valueuom')].reset_index(drop=True)

In [5]:
# check if df has result value that can not be type conversed to float.
df.iloc[:, -2].astype('float')
# we dont have such columns

0            86.0
1           151.0
2            90.0
3            18.0
4             4.0
            ...  
24348860     87.0
24348861     24.0
24348862     98.6
24348863    143.0
24348864    107.0
Name: valuenum, Length: 24348865, dtype: float64

In [6]:
# missing value handling
print(df.loc[pd.isna(df.valuenum)].itemid.value_counts())
df = df.drop(pd.isna(df.valuenum).index[pd.isna(df.valuenum)]).reset_index(drop=True)
# no missing value

Series([], Name: itemid, dtype: int64)


In [7]:
# unit conversion
# check which item may need unit conversion 
d_predictor['items'][~pd.isna(d_predictor.CE)].ravel()

array(['Heart rate', 'Respiratory rate', 'Temperature', 'SBP', 'DBP',
       'CVP', 'PaO2', 'FiO2', 'GCS', 'Bilirubin', 'Platelets',
       'Creatinine', 'Lactate', 'BUN', 'Arterial pH', 'WBC', 'PaCO2',
       'Hemoglobin', 'Hematocrit', 'Potassium', 'Sodium',
       'C Reactive Protein', 'Weight', 'Height'], dtype=object)

In [8]:
# heart rate
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][0]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

bpm    3880579
Name: valueuom, dtype: int64


count    3.880579e+06
mean     9.135713e+01
std      5.079131e+03
min     -2.413950e+05
0.1%     3.800000e+01
1%       5.200000e+01
5%       6.000000e+01
50%      8.800000e+01
95%      1.210000e+02
99%      1.370000e+02
99.9%    1.610000e+02
max      1.000000e+07
Name: valuenum, dtype: float64

In [9]:
# respatory rate 
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][1]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

insp/min    4296278
Name: valueuom, dtype: int64


count    4.296278e+06
mean     2.150436e+01
std      1.142256e+03
min     -1.000000e+00
0.1%     0.000000e+00
1%       8.000000e+00
5%       1.200000e+01
50%      2.000000e+01
95%      3.200000e+01
99%      3.800000e+01
99.9%    4.872300e+01
max      2.355560e+06
Name: valuenum, dtype: float64

In [10]:
# Temperature
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][2]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# we should convert the unit F to C
tmp_cond = df.itemid.isin([tmp_itemid[0]]) # F
df.loc[tmp_cond, 'valuenum'] = np.array((df.valuenum[tmp_cond]-32)*5/9)
df.loc[tmp_cond, 'valueuom'] = '°C'
df.loc[tmp_cond, 'itemid'] = tmp_itemid[1]
# check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
tmp_cond = df.itemid.isin(tmp_itemid)
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

°F    938609
°C    156190
Name: valueuom, dtype: int64
°C    1094799
Name: valueuom, dtype: int64


count    1.094799e+06
mean     3.707046e+01
std      3.602458e+00
min     -7.327778e+01
0.1%     3.280000e+01
1%       3.516667e+01
5%       3.594444e+01
50%      3.700000e+01
95%      3.838889e+01
99%      3.922222e+01
99.9%    4.005556e+01
max      6.490000e+02
Name: valuenum, dtype: float64

In [11]:
# SBP
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][3]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mmHg    3982108
Name: valueuom, dtype: int64


count    3.982108e+06
mean     1.186671e+02
std      5.101672e+02
min     -6.900000e+01
0.1%     4.800000e+01
1%       7.300000e+01
5%       8.600000e+01
50%      1.160000e+02
95%      1.600000e+02
99%      1.800000e+02
99.9%    2.070000e+02
max      1.003070e+06
Name: valuenum, dtype: float64

In [12]:
# DBP
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][4]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mmHg    3981338
Name: valueuom, dtype: int64


count    3.981338e+06
mean     6.302683e+01
std      2.782556e+02
min     -4.000000e+01
0.1%     2.100000e+01
1%       3.300000e+01
5%       4.100000e+01
50%      6.000000e+01
95%      8.900000e+01
99%      1.050000e+02
99.9%    1.370000e+02
max      1.141000e+05
Name: valuenum, dtype: float64

In [13]:
# CVP
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][5]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mmHg    485818
Name: valueuom, dtype: int64


count    485818.000000
mean         16.476485
std          37.057964
min         -41.000000
0.1%         -6.000000
1%            0.000000
5%            3.000000
50%          12.000000
95%          24.000000
99%         271.000000
99.9%       351.000000
max        7785.000000
Name: valuenum, dtype: float64

In [14]:
# paO2
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][6]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mmHg    206402
Name: valueuom, dtype: int64


count    206402.000000
mean        197.904977
std        8803.314368
min         -17.000000
0.1%         31.000000
1%           50.000000
5%           63.000000
50%         103.000000
95%         245.000000
99%         407.000000
99.9%       516.000000
max      999999.000000
Name: valuenum, dtype: float64

In [15]:
# fiO2
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][7]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

Series([], Name: valueuom, dtype: int64)


count    645842.000000
mean         48.021418
std          29.212437
min           0.000000
0.1%          2.000000
1%           30.000000
5%           30.000000
50%          40.000000
95%          90.000000
99%         100.000000
99.9%       100.000000
max       10050.000000
Name: valuenum, dtype: float64

In [16]:
# GCS
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][8]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])
# GCS doesnt need outlier deletion

Series([], Name: valueuom, dtype: int64)


count    2.777671e+06
mean     3.597729e+00
std      1.859882e+00
min      1.000000e+00
0.1%     1.000000e+00
1%       1.000000e+00
5%       1.000000e+00
50%      4.000000e+00
95%      6.000000e+00
99%      6.000000e+00
99.9%    6.000000e+00
max      6.000000e+00
Name: valuenum, dtype: float64

In [17]:
# bili
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][9]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mg/dL    81787
Name: valueuom, dtype: int64


count     81794.000000
mean         53.325510
std        6992.930629
min           0.000000
0.1%          0.100000
1%            0.200000
5%            0.200000
50%           1.200000
95%          21.500000
99%          38.600000
99.9%        55.200000
max      999999.000000
Name: valuenum, dtype: float64

In [18]:
# platelets
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][10]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

K/uL    235008
Name: valueuom, dtype: int64


count    235054.000000
mean        347.018252
std       12024.533324
min           0.000000
0.1%          6.000000
1%           15.000000
5%           35.000000
50%         173.000000
95%         476.000000
99%         685.000000
99.9%      1031.947000
max      999999.000000
Name: valuenum, dtype: float64

In [19]:
# creatinine
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][11]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mg/dL    264516
Name: valueuom, dtype: int64


count    264563.000000
mean         50.811621
std        7009.646603
min          -0.100000
0.1%          0.100000
1%            0.300000
5%            0.400000
50%           1.200000
95%           4.700000
99%           7.600000
99.9%        13.043800
max      999999.000000
Name: valuenum, dtype: float64

In [20]:
# lactate
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][12]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mmol/L    159189
Name: valueuom, dtype: int64


count    1.592050e+05
mean     1.425838e+02
std      1.192117e+04
min      0.000000e+00
0.1%     4.000000e-01
1%       6.000000e-01
5%       8.000000e-01
50%      1.800000e+00
95%      7.900000e+00
99%      1.430000e+01
99.9%    2.150000e+01
max      1.276100e+06
Name: valuenum, dtype: float64

In [21]:
# bun
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][13]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mg/dL    263964
Name: valueuom, dtype: int64


count    264035.000000
mean        167.981567
std       11512.262599
min           0.000000
0.1%          2.000000
1%            4.000000
5%            8.000000
50%          27.000000
95%          91.000000
99%         128.000000
99.9%       182.000000
max      999999.000000
Name: valuenum, dtype: float64

In [22]:
# arterial pH
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][14]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

units    211087
Name: valueuom, dtype: int64


count    211268.000000
mean         92.579310
std        9229.927267
min           0.000000
0.1%          6.940000
1%            7.090000
5%            7.210000
50%           7.390000
95%           7.510000
99%           7.550000
99.9%         7.610000
max      999999.000000
Name: valuenum, dtype: float64

In [23]:
# wbc
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][15]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

K/uL    229771
Name: valueuom, dtype: int64


count    229813.000000
mean        178.721353
std       12857.704237
min           0.000000
0.1%          0.100000
1%            0.800000
5%            3.900000
50%          11.500000
95%          28.400000
99%          45.100000
99.9%       121.000000
max      999999.000000
Name: valuenum, dtype: float64

In [24]:
# paco2
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][16]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mmHg    206402
Name: valueuom, dtype: int64


count    206402.000000
mean        119.203264
std        8803.778278
min           0.000000
0.1%         16.000000
1%           22.000000
5%           28.000000
50%          40.000000
95%          61.000000
99%          79.000297
99.9%       110.000000
max      999999.000000
Name: valuenum, dtype: float64

In [25]:
# hemoglobin
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][17]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

g/dl    250969
Name: valueuom, dtype: int64


count    250969.000000
mean        136.852267
std       11291.031957
min           0.000000
0.1%          4.700000
1%            6.200000
5%            7.000000
50%           9.100000
95%          12.700000
99%          14.600000
99.9%        16.900000
max      999999.000000
Name: valuenum, dtype: float64

In [26]:
# hematocrit
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][18]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

%    278036
Name: valueuom, dtype: int64


count    2.781780e+05
mean     1.507615e+02
std      1.138215e+04
min      0.000000e+00
0.1%     1.420000e+01
1%       1.910000e+01
5%       2.160000e+01
50%      2.770000e+01
95%      3.830000e+01
99%      4.410000e+01
99.9%    5.130000e+01
max      2.011200e+06
Name: valuenum, dtype: float64

In [27]:
# potassium
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][19]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mEq/L    343735
Name: valueuom, dtype: int64


count    343806.000000
mean         59.381962
std        7433.724983
min           0.000000
0.1%          2.300000
1%            2.900000
5%            3.200000
50%           4.000000
95%           5.300000
99%           6.200000
99.9%         8.100000
max      999999.000000
Name: valuenum, dtype: float64

In [28]:
# sodium
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][20]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mEq/L    284103
Name: valueuom, dtype: int64


count    284205.000000
mean        181.581584
std        6496.891197
min          -4.000000
0.1%        115.000000
1%          125.000000
5%          130.000000
50%         139.000000
95%         149.000000
99%         156.000000
99.9%       165.000000
max      999999.000000
Name: valuenum, dtype: float64

In [29]:
# CRP
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][21]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

mg/L    1770
Name: valueuom, dtype: int64


count    1770.000000
mean      113.217119
std        83.780406
min         0.100000
0.1%        0.176900
1%          1.538000
5%          5.890000
50%       100.300000
95%       266.350000
99%       300.000000
99.9%     386.997900
max       531.300000
Name: valuenum, dtype: float64

In [30]:
# Weight
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][22]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# check the distribution of the data
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

kg    160044
Name: valueuom, dtype: int64


count    160044.000000
mean         90.398089
std          26.723832
min         -95.900000
0.1%          0.000000
1%           44.500000
5%           56.400000
50%          87.400000
95%         134.400000
99%         170.000000
99.9%       285.000000
max         876.000000
Name: valuenum, dtype: float64

In [31]:
# Height
tmp_itemid = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.CE[~pd.isna(d_predictor.CE)].ravel()][23]
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
# we should convert the unit F to C
tmp_cond = df.itemid.isin([tmp_itemid[0]]) # inch
df.loc[tmp_cond, 'valuenum'] = inch_to_cm(df.valuenum[tmp_cond])
df.loc[tmp_cond, 'valueuom'] = 'cm'
df.loc[tmp_cond, 'itemid'] = tmp_itemid[1]
# check the distribution of the data
tmp_cond = df.itemid.isin(tmp_itemid)
print(df.loc[tmp_cond].valueuom.value_counts())
df.loc[tmp_cond].valuenum.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

Inch    13462
cm      13462
Name: valueuom, dtype: int64
cm    26924
Name: valueuom, dtype: int64


count    2.692400e+04
mean     3.242422e+02
std      1.806039e+04
min      0.000000e+00
0.1%     1.598060e+01
1%       1.400000e+02
5%       1.520000e+02
50%      1.700000e+02
95%      1.854200e+02
99%      1.915929e+02
99.9%    2.010000e+02
max      2.095680e+06
Name: valuenum, dtype: float64

In [32]:
# outlier handling (except urine)
outlier_criteria = pd.read_csv('processed_data/sepsis/outlier_criteria.csv')
drop_idx = []
for item in outlier_criteria.features:
    raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains(item.lower()[0:4]))&(~pd.isna(d_predictor.CE)), ('CE')].to_numpy()
    if len(raw_itemid) != 0:
        raw_itemid = raw_itemid[0]
        tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
        print(f'{item}, {tmp_itemid}')
        
        lb, ub = outlier_criteria.loc[outlier_criteria.features.str.lower().str.contains(item.lower()[0:4]), ('lb', 'ub')].to_numpy()[0]
        if item.lower()[0:4] in [i[0:4] for i in outlier_criteria.loc[outlier_criteria.criteria == '<=, >=', 'features'].str.lower()]: # <=, >=
            tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.valuenum <= lb) | (df.valuenum >= ub)))
        elif item.lower()[0:4] in [i[0:4] for i in outlier_criteria.loc[outlier_criteria.criteria == '<, >=', 'features'].str.lower()]: # <, >=
            tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.valuenum < lb) | (df.valuenum >= ub)))
        elif item.lower()[0:4] in [i[0:4] for i in outlier_criteria.loc[outlier_criteria.criteria == '<=, >', 'features'].str.lower()]: # <=, >
            tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.valuenum <= lb) | (df.valuenum > ub)))
        elif item.lower()[0:4] in [i[0:4] for i in outlier_criteria.loc[outlier_criteria.criteria == '<, >', 'features'].str.lower()]: #<, >
            tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.valuenum < lb) | (df.valuenum > ub)))
        drop_idx += tmp_cond.index[tmp_cond].to_list()

print(f'{df.shape[0]} -> {df.iloc[list(set(df.index) - set(drop_idx))].shape[0]}')
df.iloc[drop_idx].itemid.value_counts()

Heart rate, [220045]
Respiratory rate, [224690, 220210]
Temperature, [223761, 223762]
SBP, [220050, 225309, 220179]
DBP, [220051, 225310, 220180]
CVP, [220074]
PaO2, [220224]
FiO2, [223835]
Bilirubin, [225690]
Platelets, [227457]
Creatinine, [220615]
Lactate, [225668]
BUN, [225624]
Arterial pH, [223830]
WBC, [220546]
PaCO2, [220235]
Hemoglobin, [220228]
Hematocrit, [226540, 220545]
Potassium, [227442, 227464]
Sodium, [220645]
Height, [226707, 226730]
Weight, [224639]
24348865 -> 24313002


220210    16107
220074    11942
223835     1582
220045     1510
223762     1171
224690      965
220051      548
220180      488
224639      325
220224      320
220050      134
220179      120
220546      114
225310       90
220228       52
220545       46
227457       46
225624       38
225690       34
220235       31
227464       29
220645       28
223830       27
227442       25
225668       23
226540       22
225309       21
220615       17
226730        8
Name: itemid, dtype: int64

In [33]:
df.drop(drop_idx).to_csv('processed_data/sepsis/CE_R.csv', index=False)

## 4. inputevents processing

In [2]:
df = pd.read_csv(dirs[3])
df.starttime = pd.to_datetime(df.starttime) # for time should be able to calculated
df.endtime = pd.to_datetime(df.endtime) # for time should be able to calculated

In [3]:
# from the d_predictor, choose the item ids for labevents
d_predictor = pd.read_csv('processed_data/sepsis/d_predictors.csv')
tmp = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.IE[~pd.isna(d_predictor.IE)].ravel()]
itemids_IE = []
for i in tmp: 
    itemids_IE += i

In [4]:
# filter data to have only predictor relevant rows and relevent columns
tmp_cond = df.itemid.isin(itemids_IE)
df = df.loc[tmp_cond, ('subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'itemid', 'amount', 'amountuom', 'rate', 'rateuom', 'patientweight')].reset_index(drop=True)
# 6179964 -> 3409268

In [5]:
# check if df has result value that can not be type conversed to float.
df.loc[:, 'amount'].astype('float')
df.loc[:, 'rate'].astype('float')
df.loc[:, 'patientweight'].astype('float')
# we dont have such columns

0          71.2
1          71.2
2          71.2
3          71.2
4          71.2
           ... 
3409263    94.0
3409264    94.0
3409265    94.0
3409266    94.0
3409267    94.0
Name: patientweight, Length: 3409268, dtype: float64

In [6]:
# missing value handling for only amount, because fluid or patient weight can have nan rate.
print(df.loc[pd.isna(df.amount)].itemid.value_counts())
df = df.drop(pd.isna(df.amount).index[pd.isna(df.amount)]).reset_index(drop=True)
# no missing value

Series([], Name: itemid, dtype: int64)


In [7]:
# unit conversion
# check which item may need unit conversion 
d_predictor['items'][~pd.isna(d_predictor.IE)].ravel()

array(['Epinephrine', 'Dopamine', 'Dobutamine', 'Norepinephrine',
       'Phenylephrine', 'Vasopressin', 'Fluid'], dtype=object)

In [8]:
print(df.amountuom.value_counts())
print(df.rateuom.value_counts())
# for amount, we will try to use unit as ml and mg
# for rate of vasopressor, we will try to use mcg/kg/min

ml       2926487
mg        460946
units      21796
L             35
mcg            3
cm3            1
Name: amountuom, dtype: int64
mL/hour       1654125
mcg/kg/min     460947
units/hour      21793
mL/min            135
units/min           3
mg/kg/min           2
Name: rateuom, dtype: int64


In [9]:
# Epinephrine
raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains('epin'.lower()[0:4]))&(~pd.isna(d_predictor.IE)), ('IE')].to_numpy()[0]
tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
tmp_cond = df.itemid.isin(tmp_itemid)
# amount unit check
print(df.loc[tmp_cond].amountuom.value_counts())
# amount distribution check
print(df.loc[tmp_cond].amount.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# rate unit check
print(df.loc[tmp_cond].rateuom.value_counts())
# rate distribution check
print(df.loc[tmp_cond].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))

# convert mcg to mg
tmp_cond = (df.itemid.isin(tmp_itemid)) & (df.amountuom == 'mcg')
df.loc[tmp_cond, 'amount'] = mcg_to_mg(df.loc[tmp_cond, 'amount'])
df.loc[tmp_cond, 'amountuom'] = 'mg'

df.loc[tmp_cond, 'amount'].describe()

mg     14584
mcg        3
Name: amountuom, dtype: int64
count    14587.000000
mean         1.180257
std          3.317072
min          0.000112
0.1%         0.001339
1%           0.005483
5%           0.019017
50%          0.380539
95%          6.354799
99%          8.000000
99.9%       16.000001
max        234.920636
Name: amount, dtype: float64
mcg/kg/min    14587
Name: rateuom, dtype: int64
count    14587.000000
mean         0.181074
std          0.564599
min          0.001000
0.1%         0.005001
1%           0.009993
5%           0.010014
50%          0.060027
95%          0.765298
99%          2.042211
99.9%        3.548371
max         41.142862
Name: rate, dtype: float64


count    3.000000
mean     0.165291
std      0.092031
min      0.060952
25%      0.130476
50%      0.200000
75%      0.217460
max      0.234921
Name: amount, dtype: float64

In [10]:
# Dopamin
raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains('dopa'.lower()[0:4]))&(~pd.isna(d_predictor.IE)), ('IE')].to_numpy()[0]
tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
tmp_cond = df.itemid.isin(tmp_itemid)
# amount unit check
print(df.loc[tmp_cond].amountuom.value_counts())
# amount distribution check
print(df.loc[tmp_cond].amount.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# rate unit check
print(df.loc[tmp_cond].rateuom.value_counts())
# rate distribution check
print(df.loc[tmp_cond].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# no need to convert units

mg    10850
Name: amountuom, dtype: int64
count    10850.000000
mean        88.824006
std        126.117236
min          0.017602
0.1%         0.146699
1%           0.639752
5%           2.015422
50%         33.048929
95%        395.840018
99%        400.000098
99.9%      800.000003
max        801.663511
Name: amount, dtype: float64
mcg/kg/min    10850
Name: rateuom, dtype: int64
count    10850.000000
mean         8.486185
std         52.284165
min          0.200020
0.1%         0.500050
1%           1.000241
5%           2.000000
50%          5.962188
95%         19.975521
99%         20.307962
99.9%      119.018174
max       4000.000000
Name: rate, dtype: float64


In [11]:
# Dobutamin
raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains('dobu'.lower()[0:4]))&(~pd.isna(d_predictor.IE)), ('IE')].to_numpy()[0]
tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
tmp_cond = df.itemid.isin(tmp_itemid)
# amount unit check
print(df.loc[tmp_cond].amountuom.value_counts())
# amount distribution check
print(df.loc[tmp_cond].amount.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# rate unit check
print(df.loc[tmp_cond].rateuom.value_counts())
# rate distribution check
print(df.loc[tmp_cond].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# no need to convert units

mg    6200
Name: amountuom, dtype: int64
count    6200.000000
mean      145.686638
std       137.140404
min         0.014964
0.1%        0.187120
1%          0.600091
5%          2.704009
50%       117.054321
95%       460.631039
99%       500.000011
99.9%     999.120445
max      1000.000037
Name: amount, dtype: float64
mcg/kg/min    6200
Name: rateuom, dtype: int64
count    6200.000000
mean        5.228331
std         4.121954
min         0.080024
0.1%        0.300017
1%          0.500152
5%          1.500093
50%         4.999570
95%        10.348368
99%        20.030847
99.9%      32.473437
max       135.869563
Name: rate, dtype: float64


In [12]:
# Norepinephrine
raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains('nore'.lower()[0:4]))&(~pd.isna(d_predictor.IE)), ('IE')].to_numpy()[0]
tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
tmp_cond = df.itemid.isin(tmp_itemid)
# amount unit check
print(df.loc[tmp_cond].amountuom.value_counts())
# amount distribution check
print(df.loc[tmp_cond].amount.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# rate unit check
print(df.loc[tmp_cond].rateuom.value_counts())
# rate distribution check
print(df.loc[tmp_cond].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# need to convert unit of rate

tmp_cond = (df.itemid.isin(tmp_itemid)) & (df.rateuom =='mg/kg/min')
df.loc[tmp_cond, 'rate'] = np.array(df.loc[tmp_cond].rate*1000)
df.loc[tmp_cond, 'rateuom'] = 'mcg/kg/min'

print(df.loc[(df.itemid.isin(tmp_itemid))].rateuom.value_counts())
print(df.loc[(df.itemid.isin(tmp_itemid))].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))

mg    286862
Name: amountuom, dtype: int64
count    286862.000000
mean          1.240004
std           2.130145
min           0.000021
0.1%          0.002106
1%            0.008902
5%            0.031086
50%           0.452874
95%           5.668001
99%           9.754658
99.9%        16.000001
max          36.727881
Name: amount, dtype: float64
mcg/kg/min    286860
mg/kg/min          2
Name: rateuom, dtype: int64
count    286862.000000
mean          0.158394
std           0.895220
min           0.000200
0.1%          0.009969
1%            0.010059
5%            0.020031
50%           0.100110
95%           0.407393
99%           0.514064
99.9%         2.025534
max         359.550595
Name: rate, dtype: float64
mcg/kg/min    286862
Name: rateuom, dtype: int64
count    286862.000000
mean          0.158604
std           0.898712
min           0.000200
0.1%          0.009969
1%            0.010059
5%            0.020031
50%           0.100110
95%           0.407414
99%           0.514106


In [13]:
# Phenylephrine
raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains('phen'.lower()[0:4]))&(~pd.isna(d_predictor.IE)), ('IE')].to_numpy()[0]
tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
tmp_cond = df.itemid.isin(tmp_itemid)
# amount unit check
print(df.loc[tmp_cond].amountuom.value_counts())
# amount distribution check
print(df.loc[tmp_cond].amount.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# rate unit check
print(df.loc[tmp_cond].rateuom.value_counts())
# rate distribution check
print(df.loc[tmp_cond].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# need to convert unit of rate

mg    142450
Name: amountuom, dtype: int64
count    142450.000000
mean         11.852782
std          24.738890
min          -0.474000
0.1%          0.019759
1%            0.083810
5%            0.289041
50%           4.032292
95%          49.657203
99%         133.192845
99.9%       240.000011
max         274.464045
Name: amount, dtype: float64
mcg/kg/min    142450
Name: rateuom, dtype: int64
count    142450.000000
mean          1.603521
std           5.202831
min          -5.724638
0.1%          0.050007
1%            0.100087
5%            0.250251
50%           1.001640
95%           4.301075
99%           5.098212
99.9%        16.874231
max         880.058706
Name: rate, dtype: float64


In [14]:
# Vasopressin
raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains('vaso'.lower()[0:4]))&(~pd.isna(d_predictor.IE)), ('IE')].to_numpy()[0]
tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
tmp_cond = df.itemid.isin(tmp_itemid)
# amount unit check
print(df.loc[tmp_cond].amountuom.value_counts())
# amount distribution check
print(df.loc[tmp_cond].amount.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# rate unit check
print(df.loc[tmp_cond].rateuom.value_counts())
# rate distribution check
print(df.loc[tmp_cond].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))

# need to convert unit of rate
tmp_cond = (df.itemid.isin(tmp_itemid)) & (df.rateuom =='units/min')
df.loc[tmp_cond, 'rate'] = np.array(df.loc[tmp_cond].rate*60)
df.loc[tmp_cond, 'rateuom'] = 'units/hour'
print(df.loc[(df.itemid.isin(tmp_itemid))].rateuom.value_counts())
print(df.loc[(df.itemid.isin(tmp_itemid))].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))

units    21796
Name: amountuom, dtype: int64
count    21796.000000
mean        20.297584
std         20.308966
min          0.002078
0.1%         0.020000
1%           0.100000
5%           0.380886
50%         13.884924
95%         40.000002
99%         99.879998
99.9%      100.000004
max        399.999997
Name: amount, dtype: float64
units/hour    21793
units/min         3
Name: rateuom, dtype: int64
count    21796.000000
mean         2.496476
std         17.064405
min          0.016635
0.1%         0.240000
1%           0.800009
5%           1.200000
50%          2.400000
95%          3.605508
99%          4.275367
99.9%       31.356483
max       2400.000000
Name: rate, dtype: float64
units/hour    21796
Name: rateuom, dtype: int64
count    21796.000000
mean         2.497018
std         17.064392
min          0.016635
0.1%         0.247928
1%           0.800084
5%           1.200000
50%          2.400000
95%          3.605518
99%          4.280669
99.9%       31.356483
max       240

In [15]:
# Fluid
raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains('fluid'.lower()[0:4]))&(~pd.isna(d_predictor.IE)), ('IE')].to_numpy()[0]
tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
tmp_cond = df.itemid.isin(tmp_itemid)
# amount unit check
print(df.loc[tmp_cond].amountuom.value_counts())
# amount distribution check
print(df.loc[tmp_cond].amount.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))
# rate unit check
print(df.loc[tmp_cond].rateuom.value_counts())
# rate distribution check
print(df.loc[tmp_cond].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))

# need to convert unit of amount and rate
tmp_cond = (df.itemid.isin(tmp_itemid)) & (df.amountuom =='cm3')
df.loc[tmp_cond, 'amountuom'] = 'ml'
tmp_cond = (df.itemid.isin(tmp_itemid)) & (df.amountuom =='L')
df.loc[tmp_cond, 'amount'] = np.array(df.loc[tmp_cond, 'amount']*1000)
df.loc[tmp_cond, 'amountuom'] = 'ml'

tmp_cond = (df.itemid.isin(tmp_itemid)) & (df.rateuom == 'mL/min')
df.loc[tmp_cond, 'rate'] = np.array(df.loc[tmp_cond].rate*60)
df.loc[tmp_cond, 'rateuom'] = 'mL/hour'
print(df.loc[(df.itemid.isin(tmp_itemid))].rateuom.value_counts())
print(df.loc[(df.itemid.isin(tmp_itemid))].rate.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]]))

ml     2926487
L           35
cm3          1
Name: amountuom, dtype: int64
count    2.926523e+06
mean     1.255659e+02
std      2.975340e+02
min     -1.058330e+01
0.1%     4.260528e-02
1%       4.952292e-01
5%       2.301716e+00
50%      5.000000e+01
95%      5.000000e+02
99%      1.000000e+03
99.9%    3.500000e+03
max      1.228990e+05
Name: amount, dtype: float64
mL/hour    1654125
mL/min         135
Name: rateuom, dtype: int64
count    1.654260e+06
mean     4.485794e+01
std      1.716677e+02
min     -1.185000e+02
0.1%     4.973477e-01
1%       7.000369e-01
5%       2.000000e+00
50%      1.615557e+01
95%      1.500000e+02
99%      5.000000e+02
99.9%    9.999999e+02
max      6.000000e+04
Name: rate, dtype: float64
mL/hour    1654260
Name: rateuom, dtype: int64
count    1.654260e+06
mean     4.501024e+01
std      1.750986e+02
min     -1.185000e+02
0.1%     4.985045e-01
1%       7.001764e-01
5%       2.000000e+00
50%      1.615713e+01
95%      1.500000e+02
99%      5.000000e+02
99.9%   

In [21]:
# outlier handling for patient weight
lb = 0
ub = 250

tmp_cond = (df.patientweight < lb) | (df.patientweight > ub)
drop_idx = tmp_cond.index[tmp_cond].to_list()

df.loc[drop_idx].patientweight.describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])

df.loc[drop_idx, 'patientweight'] = np.nan

In [22]:
# outlier handling
drop_idx = []
for item in d_predictor.loc[(~pd.isna(d_predictor.IE)), ('items')].tolist():
    raw_itemid = d_predictor.loc[(d_predictor['items'].str.lower().str.contains(item.lower()[0:4]))&(~pd.isna(d_predictor.IE)), ('IE')].to_numpy()[0]
    tmp_itemid = np.array(raw_itemid.split(',')).astype('int').tolist() if (str(raw_itemid).__contains__(',')) else [int(raw_itemid)]
    print(f'{item}, {tmp_itemid}')
    
    lb = 0 
    ub = df.loc[df.itemid.isin(tmp_itemid), 'amount'].describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])['99.9%']
    tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.amount < lb) | (df.amount > ub)))
    drop_idx += tmp_cond.index[tmp_cond].to_list()

    ub = df.loc[df.itemid.isin(tmp_itemid), 'rate'].describe(percentiles=[i/100 for i in [0.1, 1, 5, 95, 99, 99.9]])['99.9%']
    tmp_cond = ((df.itemid.isin(tmp_itemid)) & ((df.rate < lb) | (df.rate > ub)))
    drop_idx += tmp_cond.index[tmp_cond].to_list()

print(f'{df.shape[0]} -> {df.iloc[list(set(df.index) - set(drop_idx))].shape[0]}')
df.iloc[list(set(drop_idx))].itemid.value_counts()

Epinephrine, [221289]
Dopamine, [221662]
Dobutamine, [221653]
Norepinephrine, [221906]
Phenylephrine, [221749, 229630, 229632]
Vasopressin, [222315]
Fluid, [220949, 220950, 220952, 225158, 225159, 225161, 225828, 225797, 225799, 225823, 225825, 225827, 225830, 226089, 225941, 225943, 225944, 226361, 226363, 226364, 226375, 226377, 226452, 226453, 227533, 228140, 228141, 228142, 228341, 220955, 220967, 220968, 220953]
3409268 -> 3403942


226361    1679
226364     938
225158     789
221906     557
225828     337
220949     284
221749     263
225943     131
226375      70
222315      44
226089      40
221289      28
225823      25
221662      22
225159      20
226363      19
225944      17
229630      17
221653      13
225799       7
226452       7
225161       6
229632       5
225825       3
226453       2
225827       2
220950       1
Name: itemid, dtype: int64

In [23]:
df.drop(drop_idx).to_csv('processed_data/sepsis/IE_R.csv', index=False)

## 5. outputevents processing

In [2]:
df = pd.read_csv(dirs[4])
df.charttime = pd.to_datetime(df.charttime) # for time should be able to calculated
df.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'charttime', 'storetime', 'itemid',
       'value', 'valueuom'],
      dtype='object')

In [3]:
# from the d_predictor, choose the item ids for labevents
d_predictor = pd.read_csv('processed_data/sepsis/d_predictors.csv')
tmp = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.OE[~pd.isna(d_predictor.OE)].ravel()]
itemids_OE = []
for i in tmp: 
    itemids_OE += i

In [4]:
# filter data to have only predictor relevant rows and relevent columns
tmp_cond = df.itemid.isin(itemids_OE)
df = df.loc[tmp_cond, ('subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value', 'valueuom')].reset_index(drop=True)

In [5]:
# check if df has result value that can not be type conversed to float.
df.iloc[:, -2].astype('float')
# we dont have such columns

0          100.0
1          120.0
2          300.0
3          600.0
4          275.0
           ...  
2050470     80.0
2050471    100.0
2050472     55.0
2050473    120.0
2050474    325.0
Name: value, Length: 2050475, dtype: float64

In [6]:
# missing value handling for only amount, because fluid can have nan rate
print(df.loc[pd.isna(df.value)].itemid.value_counts())
# no missing value

Series([], Name: itemid, dtype: int64)


In [7]:
# unit conversion
print(df.valueuom.value_counts())
# no need for unit conversion

ml    2050475
Name: valueuom, dtype: int64


In [8]:
# 227489 - 227488 is the real urine output
tmp_itemid = [227488, 227489]

df = df.sort_values(['stay_id', 'charttime', 'itemid'])

cnt = 0
tmp_idx = []
for stay in tqdm(df.stay_id.unique()):
    tmp_cond = (df.stay_id == stay) & (df.itemid.isin(tmp_itemid))
    part_df = df.loc[tmp_cond]

    for idx, ct in enumerate(part_df.charttime):
        if len(part_df.loc[(part_df.charttime == ct)].itemid) == 2:
            cnt += 1
        else:
            tmp_idx.append(part_df.index[idx])

print(cnt/sum(df.itemid.isin(tmp_itemid)))
df.loc[tmp_idx]

100%|████████████████████████████████████████████████████████████████████████████| 25494/25494 [04:26<00:00, 95.56it/s]


0.9046454767726161


,subject_id,hadm_id,stay_id,charttime,itemid,value,valueuom
296316,11442770,28587354,30174349,2149-05-02 02:00:00,227488,10.0,ml
1720600,18386668,29019639,30181943,2110-06-22 18:00:00,227488,165.0,ml
1720612,18386668,29019639,30181943,2110-06-23 06:00:00,227488,600.0,ml
1526886,17414351,23018977,30254621,2179-09-14 00:23:00,227488,20.0,ml
1035626,15080981,23329122,30333231,2129-06-14 16:27:00,227488,70.0,ml
...,...,...,...,...,...,...,...
683211,13364829,26673737,39923560,2208-01-01 08:00:00,227488,3000.0,ml
683212,13364829,26673737,39923560,2208-01-01 08:16:00,227489,3225.0,ml
683215,13364829,26673737,39923560,2208-01-01 10:00:00,227489,0.0,ml
683218,13364829,26673737,39923560,2208-01-01 13:00:00,227488,3000.0,ml


### main issues in irrigant out - in urine output processing
- irrigant out and in is not recorded in the same charttime
- irrigant out or in is recorded consecutively without each other item`s interruption
- how to solve it?
1. deal with the second problem by concat the information to the latest timebucket
2. the first problem can be handled by subtacting the latest irrigant in item value from the current irrigant out value. 

In [26]:
# 227489 - 227488 : step 1
# consecutive record checking and concating

drop_idx = []
concat_data = []
for stay in tqdm(df.stay_id.unique()):
    tmp_cond = (df.stay_id == stay) & (df.itemid.isin(tmp_itemid))
    part_df = df.loc[tmp_cond]
    
    # check for consecutiveness
    for idx, df_idx in enumerate(part_df.index):
        if df_idx == part_df.index[0]:
            tmp_idx = []
            continue
        
        if part_df.loc[df_idx, 'itemid'] == part_df.loc[part_df.index[idx-1], 'itemid']:
            tmp_idx += [part_df.index[idx-1], df_idx]
            
        elif len(tmp_idx) != 0:
            tmp_idx = list(set(tmp_idx))
            drop_idx += tmp_idx
            tmp = part_df.loc[part_df.index[idx-1]].tolist()
            tmp[6] = sum(part_df.loc[tmp_idx, 'value'])
            concat_data.append(tmp)
            tmp_idx = []

100%|████████████████████████████████████████████████████████████████████████████| 25494/25494 [04:17<00:00, 98.88it/s]


In [28]:
print(df.shape[0])
df = pd.concat([df.drop(drop_idx), pd.DataFrame(concat_data, columns = df.columns)], axis = 0).sort_values(['stay_id', 'charttime', 'itemid'])
df

2050475


,subject_id,hadm_id,stay_id,charttime,itemid,value,valueuom
506950,12466550,23998182,30000153,2174-09-29 12:12:00,226559,280.0,ml
506951,12466550,23998182,30000153,2174-09-29 14:00:00,226559,45.0,ml
506952,12466550,23998182,30000153,2174-09-29 14:56:00,226627,100.0,ml
506953,12466550,23998182,30000153,2174-09-29 15:00:00,226559,50.0,ml
506954,12466550,23998182,30000153,2174-09-29 16:00:00,226559,50.0,ml
...,...,...,...,...,...,...,...
739035,13651601,22584645,39999230,2147-09-09 08:00:00,226559,80.0,ml
739036,13651601,22584645,39999230,2147-09-09 10:00:00,226559,100.0,ml
739037,13651601,22584645,39999230,2147-09-09 12:00:00,226559,60.0,ml
739038,13651601,22584645,39999230,2147-09-09 14:00:00,226559,80.0,ml


In [29]:
df = df.reset_index(drop = True)

In [30]:
# 227489 - 227488 : step 2
tmp_itemid = [227488, 227489]

for stay in tqdm(df.stay_id.unique()):
    tmp_cond = (df.stay_id == stay) & (df.itemid.isin(tmp_itemid))
    part_df = df.loc[tmp_cond]

    for idx, ct in enumerate(part_df.charttime[part_df.itemid == tmp_itemid[1]]):
        tmp_cond = (part_df.charttime == max(part_df.charttime[part_df.charttime <= ct])) & (part_df.itemid == tmp_itemid[0])
        if sum(tmp_cond) == 0:
            ir_in = 0
        else:
            ir_in = part_df.loc[tmp_cond, 'value']
        
        tmp_cond = (part_df.charttime == ct) & (part_df.itemid == tmp_itemid[1])
        ir_out = part_df.loc[tmp_cond, 'value']
        
        tmp_cond = (df.charttime == ct) & (df.itemid == tmp_itemid[1])
        df.loc[tmp_cond, 'value'] = ir_out - ir_in
    
tmp_cond = (df.itemid == 227488)
df = df.drop(tmp_cond.index[tmp_cond]).reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████| 25494/25494 [05:10<00:00, 82.05it/s]


In [31]:
df.to_csv('processed_data/sepsis/OE_R.csv', index=False)
# when concatenation, urine output sholud be concatenated underthe priority rule.
# urine output from hospital should be concated when there is no urine output from icu

## 6. procedureevents processing

In [33]:
df = pd.read_csv(dirs[5])
df.starttime = pd.to_datetime(df.starttime) # for time should be able to calculated
df.endtime = pd.to_datetime(df.endtime) # for time should be able to calculated

In [35]:
# from the d_predictor, choose the item ids for labevents
d_predictor = pd.read_csv('processed_data/sepsis/d_predictors.csv')
tmp = [np.array(i.split(',')).astype('int').tolist() if (str(i).__contains__(',')) else [int(i)] for i in d_predictor.PE[~pd.isna(d_predictor.PE)].ravel()]
itemids_PE = []
for i in tmp: 
    itemids_PE += i

In [37]:
df.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'storetime',
       'itemid', 'value', 'valueuom', 'location', 'locationcategory',
       'orderid', 'linkorderid', 'ordercategoryname',
       'ordercategorydescription', 'patientweight', 'isopenbag',
       'continueinnextdept', 'statusdescription', 'originalamount',
       'originalrate'],
      dtype='object')

In [39]:
# filter data to have only predictor relevant rows and relevent columns
tmp_cond = df.itemid.isin(itemids_PE)
print(df.shape[0])
df = df.loc[tmp_cond, ('subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'itemid', 'value')].reset_index(drop=True)
print(df.shape[0])# 369551 -> 16606

369551
16606


In [40]:
# check if df has result value that can not be type conversed to float.
df.loc[:, 'value'].astype('float')
# we dont have such columns

0         2220.0
1          390.0
2         9465.0
3         6576.0
4        12640.0
          ...   
16601     8406.0
16602     4269.0
16603     1381.0
16604     1302.0
16605      240.0
Name: value, Length: 16606, dtype: float64

In [43]:
# missing value handling 
print(df.loc[pd.isna(df.value)].itemid.value_counts())
# no missing value

Series([], Name: itemid, dtype: int64)


In [44]:
df.to_csv('processed_data/sepsis/PE_R.csv', index=False)

# EON